![Go Fast](matthew-brodeur-eJ9mX6yEbAw-unsplash.jpg)

In my [last post](https://sethchart.com/posts/10/) I described how to sort a list of integers using a slight variation of selection sort, which I dubbed Zombie Bloodbath sort.
In that post, we found out that the resulting algorithm had quadratic worst-case time complexity.
It turns out that we can sort a list faster, but we need to be a little tricky. 

## What You Will Learn

In this post, I will describe how we can modify our approach to sorting to achieve optimal worst-case time complexity.
In other words, I will show you how to go fast.

## Recap from Last Time
In my last post, I decided that I would remember the algorithm more easily if I made by description melodramatic.
That worked for me, if it didn't work for you, sorry not sorry.

Sorting a list is all about comparisons between pairs of elements, to add drama these are fights to the death.
Last time, we wrote the following function that implements a fight to the death between two integers where the larger integer always wins and ties go to the integer on the left.

In [1]:
def fight(a: int, b: int) -> int:
    """This function implements a fight to the death between two integers a 
    and b and returns the winner.
    """
    if a >= b:
        winner = a
    else:
        winner = b
    return winner

assert fight(-10, 5) == 5, "Negative ten is unexpectedly scrappy!"
assert fight(7, 5) == 7, "Poor show seven, better luck next time..."
assert fight(7, 7) == 7, "How did you manage to get that wrong?"

Last time, we ran a sequence of smaller and smaller tournaments to sort our list.
This time we are going to organize our fights differently so that we can use substantially fewer fights to sort the list.

## Divide and Conquer
Imagine that we have our motley crew of integers ready to battle for sorted supremacy. 
In other words, an unsorted list of integers if you want to be boring about it.

```python 
unsorted = [4, -9, 3, 1, 5, 7, -1, 0]
```


### A little magic trick
This time I am going to do something tricky.
I am going to do a magic step, ask you to ignore it for a while, and then we will realize that we have solved our problem and no magic is needed.

We are going to split the contestants into two teams, `A` and `B`, of the same size.

```python
A = [4, -9, 3, 1]
B = [5, 7, -1, 0]
```

Here is the __magic step__.
I sort my teams so left-to-right largest-to-smallest.
I am not going to tell you how, that's magic for you baby.

```python
A = [4, 3, 1, -9]
B = [7, 5, 0, -1]
```
Right now your [bogosity](https://www.merriam-webster.com/dictionary/bogosity) meter should be in the red, but trust me, everything is fine. 
Suspend your disbelief for a little bit and I promise I will get rid of the magic by the end of this post. 

### Merging

We would be done if we could merge those two sorted teams together like shuffling a deck of cards. 
We are going to fight numbers to figure out how to merge the teams.
To merge the teams efficiently, we need to take advantage of the fact that the teams are sorted. 

Because my teams are sorted already, I know that the left most contestant is the largest in its respective team.
That means that the largest number in the two teams must be one of the two left most contestants.
If those two contestants fight, then the winner of the fight must be the largest out of all of the contestants.

### Merging by hand

Let's set up our merge.
We start with our two sorted teams and an empty hall of champions.

```python
hall_of_champions = []
A = [4, 3, 1, -9]
B = [7, 5, 0, -1]
```

To determine the largest number in the contest, I just need one fight between four and seven.
Seven wins and goes to the hall of champions, which leaves me with the teams below for the next step of merging.

```python
# After first step of merge.
hall_of_champions = [7]
A = [4, 3, 1, -9]
B = [5, 0, -1]
```

The great thing is that these two teams are still sorted.
To determine who goes to the hall of champions next, I can just have another fight between the top competitors.
In this case, four and five.

```python
# After second step of merge.
hall_of_champions = [7, 5]
A = [4, 3, 1, -9]
B = [0, -1]
```

Rinse, wash, and repeat.

Four fights zero.

```python
# After third step of merge.
hall_of_champions = [7, 5, 4]
A = [3, 1, -9]
B = [0, -1]
```

Three fights zero.
```python
# After fourth step of merge.
hall_of_champions = [7, 5, 4, 3]
A = [1, -9]
B = [0, -1]
```

One fights zero.
```python
# After fifth step of merge.
hall_of_champions = [7, 5, 4, 3, 1]
A = [-9]
B = [0, -1]
```

Negative nine fights zero.
```python
# After sixth step of merge.
hall_of_champions = [7, 5, 4, 3, 1, 0]
A = [-9]
B = [-1]
```

Negative nine fights negative one.
```python
# After seventh step of merge.
hall_of_champions = [7, 5, 4, 3, 1, 0, -1]
A = [-9]
B = []
```

No fights left, the remaining team goes to the hall of champions in order.
```python
# Done.
hall_of_champions = [7, 5, 4, 3, 1, 0, -1, -9]
A = []
B = []
```

### Merging fight-by-fight
Alright, lets write some code to see if we can walk through merging our lists fight by fight.
The key points are:
1. We always run a fight between the largest remaining contestants from our two teams. The winner goes to the hall of champions.
2. If we exhaust all of the contestants from one team, then we can send all of the remaining contestants to the hall of champions. They are sorted since they all come from the same team. Also, they are smaller than (or equal to) all of the contestants in the hall of champions since those contestants are all either larger elements from their team, or contestants from the other team who beat a larger contestant from their team.

In [15]:
from typing import List, Tuple
MergeTuple = Tuple[List[int], List[int], List[int]]
def merge_fight(merge_tuple: MergeTuple) -> MergeTuple:
    """This function takes a tuple consisting of two sorted lists A and B that are
    being merged into a third list called the hall of champions, exicutes the next 
    merge fight and updates the tuple."""
    # Unpack the lists from the input tuple.
    A = merge_tuple[0]
    B = merge_tuple[1]
    hall_of_champions = merge_tuple[2]
    # Make sure that A in a non-empty list and assign the largest element from A to a.
    try:
        a = A[0]
        #Make sure that B is a non-empty list and assign the largest element from B to b.
        try:
            b = B[0]
            # Fight a and b to determine which one goes to the hall of champions.
            winner = fight(a, b)
            # If a wins drop it from A and append to the hall of champions.
            if a == winner:
                A = A[1:]
                hall_of_champions.append(a)
            # If b wins drop it from B and append to the hall of champions.
            elif b == winner:
                B = B[1:]
                hall_of_champions.append(b)
        #If B is empty, extend the hall of champions with the remaining elements of A
        #and remove them from A.
        except IndexError:
            hall_of_champions.extend(A)
            A = []
    #If A is empty, extend the hall of champions with the remaining elements of B
    #and remove them from B.
    except IndexError:
        hall_of_champions.extend(B)
        B=[]
    #Return the updated MergeTuple
    return A, B, hall_of_champions

# This just tests the output from our function against our bare hands merge from
# last section.
A = [4, 3, 1, -9]
B = [7, 5, 0, -1]
hoc = [7, 5, 4, 3, 1, 0, -1, -9]
assert merge_fight((A, B, [])) == (A, B[1:], hoc[:1]), "Failed first step."
assert merge_fight((A, B[1:], hoc[:1])) == (A, B[2:], hoc[:2]), "Failed second step."
assert merge_fight((A, B[2:], hoc[:2])) == (A[1:], B[2:], hoc[:3]), "Failed third step."
assert merge_fight((A[1:], B[2:], hoc[:3])) == (A[2:], B[2:], hoc[:4]), "Failed fourth step."
assert merge_fight((A[2:], B[2:], hoc[:4])) == (A[3:], B[2:], hoc[:5]), "Failed fifth step."
assert merge_fight((A[3:], B[2:], hoc[:5])) == (A[3:], B[3:], hoc[:6]), "Failed sixth step."
assert merge_fight((A[3:], B[3:], hoc[:6])) == (A[3:], [], hoc[:7]), "Failed seventh step."
assert merge_fight((A[3:], [], hoc[:7])) == ([], [], hoc), "Failed last step."

Great, that worked!
The function is kind of a mess, but it worked.

### Merging two lists

Merging the lists step-by-step was a nice check, but I would rather have a function that takes two sorted lists and returns a merged list.
Let's see if we can make that happen and maybe clean up the code a little in the process.

In [16]:
def merge_teams(A: List[int], B: List[int]) -> List[int]:
    """This function takes two sorted teams, A and B, and merges them into a sorted
    hall of champions.
    """
    hall_of_champions = []
    while len(A) > 0 and len(B) > 0:
        winner = fight(A[0], B[0])
        hall_of_champions.append(winner)
        if winner == A[0]:
            A = A[1:]
        else:
            B = B[1:]
    hall_of_champions.extend(A)
    hall_of_champions.extend(B)
    return hall_of_champions

A = [4, 3, 1, -9]
B = [7, 5, 0, -1]
hoc = [7, 5, 4, 3, 1, 0, -1, -9]
assert merge_teams(A, B) == hoc, "First test failed"

A = [5, 3, 3, 1]
B = [4, 0, -1, -2, -2]
hoc = [5, 4, 3, 3, 1, 0, -1, -2, -2]
assert merge_teams(A, B) == hoc, "Second test failed"

Fantastic, we can now merge to sorted teams!
But what about the __magic step__?
How are we going to get rid of that?

### A world without magic

Here is an obvious but useful fact.
A team with one contestant is always sorted.

When I did the magic step above and suddenly sorted my teams without telling you how, I was just being lazy.
I started with the unsorted contestants below.
```python 
unsorted = [4, -9, 3, 1, 5, 7, -1, 0]
```

Then I split them into two teams.
```python
A = [4, -9, 3, 1]
B = [5, 7, -1, 0]
```

But these teams are just lists of unsorted contestants, so split them into teams again.
For example `A` becomes the two teams below.
```python
AA = [4, -9]
AB = [3, 1]
```

Now, technically those lists are sorted, but I have no way of knowing that in general, so I am going split one more time.
For example, `AA` becomes the following two teams.
```python
AAA = [4]
AAB = [-9]
```
But each of these teams has a single contestant, so they are both sorted.
I am confident that I can merge these teams into a sorted team with two contestants.

The big trick is to keep splitting up your teams until they are sorted for free, because they each only contain one contestant, then merge teams until you have one fully sorted hall of champions.

### Merge Sort
Let's take our trick an make it into code. 
Remember the keys steps are as follows.
1. Feed in an unsorted list of contestants.
2. Process the list
 1. If there are at least two contestants, then split them into two teams, __sort them__, then merge them into a sorted list. Note that when I say __sort them__, I mean feed each team into this algorithm at Step 1.
 2. If there is just one contestant, then the list is sorted, so return it.

Step 2.A makes this a [recursive algorithm](https://en.wikipedia.org/wiki/Recursion_(computer_science)) because in the course of running the algorithm we may need to run the algorithm on a sub-problem, potentially several times.

One thing that we need to be careful with when we a recursive algorithm, is making sure that we don't call the algorithm on an infinite number of sub problems.
We are safe here because every time we hit step 2.A we are splitting the list in half and feeding it into the algorithm again.
Each consecutive time we hit Step 2.A we are feeding in a strictly shorter list of contestants.
That means we will always eventually get down to teams with one contestant, so we skip over to step 2.B and break out of the recursion. 

Time for some code.

In [40]:
def merge_sort(team: List[int]) -> List[int]:
    """This function takes a list and returns a sorted list."""    
    N = len(team)
    if N > 1:
        A = merge_sort(team[:N//2])
        B = merge_sort(team[N//2:])
        team = merge_teams(A, B)
    return team

assert merge_sort([4, -9, 3, 1, 5, 7, -1, 0]) == [7, 5, 4, 3, 1, 0, -1, -9], "First test failed"
assert merge_sort([5, 3, 3, 1, 4, 0, -1, -2, -2]) == [5, 4, 3, 3, 1, 0, -1, -2, -2], "Second test failed"

## How many fights?
Alright, so we came up with a new way to sort a list.
Personally, I think this algorithm is less obvious and harder to remember, so it had better be better than Zombie Bloodbath sort.
How many fights did we need to sort our list?

Our example list had eight contestants.
Above, we counted up the number of fights required to merge the two sorted teams with four contestants.
We counted seven fights.
But that is not all, I need to sort the two teams before I can merge them.
To sort a team with four contestants, I need to merge two sorted lists with two contestants.
If you work through merging `AA` and `AB` by hand like we did for `A` and `B`, you will find that you need three fights to merge those lists.
A similar exercise with the `B` list should give you two fights to merge the sorted teams `BA = [7, 5]` and `BB = [0, -1]`.
To ensure that all of the teams of size two were sorted, we split each one into two teams of one and merged with a single fight.

So lets see, I think I get the following for a fight total.
$$ 7 + 3 + 2 + 4*1 = 16 $$

If I had used Zombie Bloodbath, I would have used several more fights.
To be exact:
$$ \tfrac{1}{2}8^2 - \tfrac{1}{2}8 = 28 $$

### The general case
__This gets a little math heavy, feel free to skip to the conclusion.__

To figure out what is happening in general, I am going to make one piece of notation.
Let's agree that $F(k)$ means the largest possible number of fights required to sort a list of $k$ integers using merge sort.

In addition, I am going to make two observations.

First, If I split a list of $k$ integers into two sorted teams, then it will take at most $k-1$ fights to merge the two teams.
Let me justify that.
There are a total of $k$ contestants in two teams.
Every fight removes the winner from their team and sends them to the hall of champions.
If there have been $k-1$ fights already, then there is only one contestant outside the hall of champions.
Therefore, this last contestant has no one to fight and the merge is over.

Second, no fights are required to sort a list with one integer.
Using the notation, $F(1) = 0$.

To keep things from getting messy, I am going to concede one more fight than I need to and just say that merging the two sorted teams into a sorted list of $k$ integers takes no more than $k$ fights, even though we know that $k-1$ will always get the job done.  

If I put this together, I find out that the number of fights required to sort a list with $k$ integers satisfies the equation below.

$$ F(k) = k + 2 F\left(\tfrac{k}{2}\right) $$

All this is saying is that the number of fights required to sort a list with $k$ integers is the number of fights required to merge two sorted teams, plus the number of fights required to sort each of the teams, which are just lists of (approximately) $\tfrac{k}{2}$ integers. Don't forget that there are two teams, hence two times $F\left(\tfrac{k}{2}\right)$.

Here is the funny thing, I can plug $\tfrac{k}{2}$ into the left side of the equation above to get the equation below.

$$ F\left(\tfrac{k}{2}\right) = \tfrac{k}{2} + 2 F\left(\tfrac{k}{4}\right) $$

Then, I can replace the $F\left(\tfrac{k}{2}\right)$ in the top equation with the right hand side of the equation we just wrote down, which gives me the following.

$$ F(k) = k + 2 \left[\tfrac{k}{2} + 2 F\left(\tfrac{k}{4}\right)\right]$$

Distributing the two through the square brackets gives me the cleaned up version below.

$$ F(k) = k + k + 4 F\left(\tfrac{k}{4}\right) $$

So we can see that the number of fights required to sort a list of $k$ integers is the number fights required to merge the two sorted teams, plus the number of fights required to merge the two pairs of sorted sub-teams created while sorting each team, plus the number of fights required to sort the four sub-teams.



We see that merging two sorted teams costs $k$ fights and merging two pairs of sorted sub-teams also costs $k$ fights.

For the sake of explanation I actually want to make one more tweak and write $4$ as $2^2$ so that I have this final version.

$$ F(k) = 2 k + 2^2 F\left(\tfrac{k}{2^2}\right) $$

It should not surprise us that we can play this trick again to obtain the equation below.

$$ F(k) = 3 k + 2^3 F\left(\tfrac{k}{2^3}\right) $$

In fact, I can do this as many times as I want, up to a point.
Let's say that $j$ is the number of times I have done this dorky algebra trick.
Then I would have the following equation.

$$ F(k) = j k + 2^j F\left(\tfrac{k}{2^j}\right) $$

I said that I can play this trick up to a point, that point is when $2^j$ is bigger than or equal to $k$.
Then, $\tfrac{k}{2^j} \le 1$.
But we said that $F(1) = 0$.
So when $j$ gets to the point where $2^j \ge k$, we have.

That means we can keep splitting teams up until $j = \lceil \log_{2} k \rceil$. 
In other words, take the log base two of $k$ and round up to the next integer, that is $j$.
Therefore, we have the following.

$$F(k) = \lceil \log_{2} k \rceil k + 2^{\lceil \log_{2} k \rceil} F(1) = \lceil \log_{2} k \rceil k = O\left(k \log k\right) $$

## Conclusion
Merge sort is a log-linear algorithm.
In big-O notation, $O\left(k\log k\right)$. 
Log-linear is faster than quadratic.
If I wanted to sort a list of one hundred integers, I would expect Zombie Bloodbath to take no more than 10,000 fights to get the job done. 
In comparison, I would expect merge sort to take at most 665 fights to sort the same list.
The bigger the list, the bigger the savings.

## Acknowledgments
Photo by <a href="https://unsplash.com/@mrbrodeur?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Matthew Brodeur</a> on <a href="https://unsplash.com/s/photos/speed?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a>